In [2]:
import requests
import json
import re
import pandas as pd

from selectolax.parser import HTMLParser
from selenium import webdriver

In [3]:
driver = webdriver.Chrome()
driver.get(
    "https://yandex.ru/maps/2/saint-petersburg/chain/magnit/35471869327/?ll=30.368146%2C59.929569&page=6&sctx=ZAAAAAgBEAAaKAoSCRR5knTNUD5AEfTF3osv%2BE1AEhIJmQ0yychZ9T8RD9WUZB2O4z8iBgABAgMEBSgAOABAAkgBYihwcmljZXNfc25pcHBldHNfbWF4X2l0ZW1zX2luX3Jlc3BvbnNlPTUwYjZyZWFycj1zY2hlbWVfTG9jYWwvR2VvL1N1YnRpdGxlcy9OZXdTdWJ0aXRsZXNFbmFibGVkPTFiJ3JlYXJyPXNjaGVtZV9Mb2NhbC9HZW8vUHJpY2VzL0VuYWJsZWQ9MWI1cmVhcnI9c2NoZW1lX0xvY2FsL0dlby9TdWJ0aXRsZXMvUmFuZG9tUGluU3VidGl0bGVzPTFqAnJ1nQHNzEw9oAEAqAEAvQGgV2cjwgGPAamR8%2BGMAsSU8JyrBa7Wk%2FEFgLGe8AXssvvSlAP58cemlQbelqbXuAKV0cO8lQbdi8CJsgGtm6aipQKDirHThwKM9N7eBcrTp7e4Ac2cjbLgBo7Kt8MEvLKls6UCiNCdtQSq5%2FnBkAehlY3ipQT3idyZ0gXThoGYugK%2Ft6z4tQGaxqrjBenon6kEztWwp6gC6gEA8gEA%2BAEAggIWY2hhaW5faWQ6KDM1NDcxODY5MzI3KYoCAJICAJoCDGRlc2t0b3AtbWFwcw%3D%3D&sll=30.368146%2C59.929569&sspn=0.689069%2C0.253601&z=10"
)

In [4]:
magnit_shops = []

In [11]:
shop.css_first("div")

In [17]:
df = pd.read_excel("shop_addrs_fixed.xlsx")
addrs = df[df.shop == "Магнит"].address.tolist()

In [13]:
dom = HTMLParser(driver.page_source)
dom.css("ul.search-list-view__list > li")

for shop in dom.css("ul.search-list-view__list > li"):
    reviews = shop.css_first(
        "div.business-rating-with-text-view > span.a11y-hidden"
    )
    if reviews:
        reviews = reviews.text().split(" ")[0]
        
    rating = shop.css_first("span.business-rating-badge-view__rating-text")
    if rating:
        rating = re.sub(
            ",", ".", rating.text()
        )
        
    address = shop.css_first("div.search-business-snippet-view__address")
    if address:
        address = address.text()
        
    coords = shop.css_first("div")
    if coords:
        coords = coords.attributes["data-coordinates"]
    
    magnit_shops.append({
        "coords": coords,
        "address": address,
        "reviews": reviews,
        "rating": rating,
    })
    
with open("magnit_addrs.json", "w") as file:
    json.dump(magnit_shops, file)
    
df_magnit = pd.read_json("magnit_addrs.json")
df_magnit["shop"] = "Магнит"

df_magnit.drop_duplicates()

,coords,address,reviews,rating,shop
0,"30.423731,59.903028","просп. Обуховской Обороны, 48, литер А, помеще...",128.0,4.2,Магнит
1,"30.413312,59.893604","ул. Седова, 17, корп. 2, корпус 2, литера А, п...",874.0,4.0,Магнит
2,"30.416120,59.889569","ул. Невзоровой, 9, стр. 1",235.0,4.3,Магнит
3,"30.441905,59.906035","Октябрьская наб., 34, корп. 4, помещение 64-Н,...",268.0,4.1,Магнит
4,"30.429308,59.892659","ул. Бабушкина, 18, помещение 1-Н",685.0,4.0,Магнит
5,"30.420421,59.891085","ул. Ткачей, 76, литера А, помещение 11-Н",634.0,4.0,Магнит
6,"30.430969,59.890288","ул. Бабушкина, 26",475.0,4.1,Магнит
7,"30.430455,59.885666","ул. Дудко, 18, лит. А",514.0,3.9,Магнит
32,None,None,NaN,NaN,Магнит
33,"30.451406,59.892595","ул. Крыленко, 1, корп. 1, стр. 2, Санкт-Петербург",434.0,4.2,Магнит


In [724]:
df_vprok[["longitude", "latitude"]] = df_vprok["coords"].str.split(",", expand=True)

In [725]:
df_vprok = df_vprok.drop(["coords"], axis=1)

In [726]:
df_vprok = df_vprok.reset_index(drop=True)

In [728]:
df_diksi.to_excel("vprok_locations.xlsx", index=False)

In [729]:
df_main = pd.concat([df_magnit, df_vprok, df_diksi])
df_main = df_main.reset_index(drop=True)

In [731]:
df_main = df_main.drop_duplicates()

In [733]:
df_main.address = "Санкт-Петербург, " + df_main.address

In [734]:
df_main

,address,reviews,rating,shop,longitude,latitude
0,"Санкт-Петербург, Средний просп. Васильевского ...",83.0,4.2,Магнит,30.249909,59.937200
1,"Санкт-Петербург, Малый просп. Васильевского ос...",571.0,4.1,Магнит,30.240457,59.938315
2,"Санкт-Петербург, Наличная ул., 35, корп. 1, ко...",399.0,3.9,Магнит,30.232841,59.940740
3,"Санкт-Петербург, Гаванская ул., 34, литер А, п...",84.0,4.0,Магнит,30.238154,59.935557
4,"Санкт-Петербург, ул. Нахимова, 8",370.0,3.8,Магнит,30.234647,59.943310
...,...,...,...,...,...,...
2068,"Санкт-Петербург, Центральная ул., 19, г. Отрадное",506.0,4.0,Дикси,30.796557,59.767081
2069,"Санкт-Петербург, ул. Гагарина, 8, г. Отрадное",449.0,3.6,Дикси,30.804156,59.769207
2070,"Санкт-Петербург, 206, д. Гарболово",167.0,4.0,Дикси,30.495247,60.342330
2071,"Санкт-Петербург, Бронницкая ул., 24, Санкт-Пет...",NaN,NaN,Дикси,30.323656,59.914022


In [735]:
df_main.to_excel("shop_addrs_fixed.xlsx", index=False)

In [198]:
df_main = pd.read_excel("shop_addrs.xlsx")